<a href="https://colab.research.google.com/github/avi8152/Sentiment-Analysis---Major-Project/blob/main/amazon_review_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
!pip install vaderSentiment
!pip install scikit-learn
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs =  SentimentIntensityAnalyzer()

In [116]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [117]:
df = pd.read_csv('/content/amazonreview.csv')
df.head()

,label,review
0,positive,Stuning even for the non-gamer: This sound tra...
1,positive,The best soundtrack ever to anything.: I'm rea...
2,positive,Amazing!: This soundtrack is my favorite music...
3,positive,Excellent Soundtrack: I truly like this soundt...
4,positive,"Remember, Pull Your Jaw Off The Floor After He..."


In [118]:
import nltk
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stopword_list]
  filtered_text = '  '.join(filtered_tokens)
  return filtered_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [89]:
def html_tag(text):
  soup = BeautifulSoup(text,"html.parser")
  new_text = soup.get_text()
  return new_text

In [90]:
!pip install contractions
import contractions
def con(text):
  expand = contractions.fix(text)
  return expand

In [91]:
import re
def remove_sp(text):
  pattern = r'[^A-Za-z0-9\s]+'
  text = re.sub(pattern,'',text)
  return text

In [119]:
df.review = df.review.apply(lambda x:x.lower())

df.review = df.review.apply(html_tag)

df.review = df.review.apply(con)

df.review = df.review.apply(remove_sp)

df.review = df.review.apply(remove_stopwords)


df.head()

,label,review
0,positive,stuning even nongamer sound track beautif...
1,positive,best soundtrack ever anything I reading ...
2,positive,amazing soundtrack favorite music time ha...
3,positive,excellent soundtrack truly like soundtrack...
4,positive,remember pull jaw floor hearing played g...


In [120]:
df['compound'] = df['review'].apply(lambda x: vs.polarity_scores(x)['compound'])
df['scores'] = df['review'].apply(lambda review: vs.polarity_scores(review))
df['comp_score'] = df['compound'].apply(lambda c: 'the review is positive' if c >=0 else 'the review is negative')
df.head()

,label,review,compound,scores,comp_score
0,positive,stuning even nongamer sound track beautif...,0.9062,"{'neg': 0.138, 'neu': 0.528, 'pos': 0.334, 'co...",the review is positive
1,positive,best soundtrack ever anything I reading ...,0.9628,"{'neg': 0.039, 'neu': 0.642, 'pos': 0.319, 'co...",the review is positive
2,positive,amazing soundtrack favorite music time ha...,0.9868,"{'neg': 0.079, 'neu': 0.49, 'pos': 0.431, 'com...",the review is positive
3,positive,excellent soundtrack truly like soundtrack...,0.9814,"{'neg': 0.121, 'neu': 0.479, 'pos': 0.4, 'comp...",the review is positive
4,positive,remember pull jaw floor hearing played g...,0.9831,"{'neg': 0.0, 'neu': 0.554, 'pos': 0.446, 'comp...",the review is positive


In [122]:
df.to_csv('revised_reviews.csv',index=False)

In [95]:
!pip install streamlit

In [123]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

df = pd.read_csv('/content/revised_reviews.csv')
x = df['review']
y = df['label']
st.title("REVIEW CLASSIFIER")
st.subheader('TFIFD Vectorizer')
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=0)

final = Pipeline([('vect',TfidfVectorizer()),('model',SVC())])
final.fit(x_train,y_train)
t = st.text_input("Enter a review")
message = final.predict([t])
if st.button("Predict"):
   st.title(message)
  

Overwriting app.py


In [97]:
!pip install pyngrok==4.1.1
!pip install pyngrok

In [ ]:
!streamlit run app.py 

In [ ]:
from pyngrok import ngrok
url = ngrok.connect(port='8501')
url